# Interactively find ROI and create a mask
- select ROI
- thresholding
- mask

In [1]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt

In [2]:
import itk
import itkwidgets
from itkwidgets import view, compare

In [3]:
input_filename="../images/bead_pack_artifacts_small.tif"
img = itk.imread(input_filename)
img = itk.median_image_filter(img)

In [4]:
#view(img,mode='x') #slicing over x

## Select ROI

In [5]:
myViewer = view(img,cmap=itkwidgets.cm.BrBG)
myViewer
#Now select a region of interest

Viewer(cmap=['BrBG'], geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.…

In [6]:
#These are the coordinates of your crop
myViewer.roi_region()

itkImageRegion3([0, 0, 0], [200, 200, 200])

In [7]:
#Convert itk image to numpy
np_img = itk.GetArrayFromImage(img) 

In [8]:
arr = itk.array_from_image(img)
roi = myViewer.roi_slice()
view(arr[roi], cmap=itkwidgets.cm.PrGn, annotations=False, ui_collapsed=True)

Viewer(annotations=False, cmap=['PRGn'], geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<…

## Thresholding: exploring threshold using mid slice

In [9]:
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, interactive, fixed, interact_manual

middle = 100
aslice = np_img[:,:,middle]
def myThresh(t=128):
    thresholded = aslice>t
    plt.imshow(thresholded, cmap='gray')
    
interact(myThresh, t=widgets.IntSlider(min=0,max=255,step=1,value=128));

interactive(children=(IntSlider(value=128, description='t', max=255), Output()), _dom_classes=('widget-interac…

In [10]:
help(itk.binary_threshold_image_filter) #todo: no relevant info

Help on function binary_threshold_image_filter in module itk.itkBinaryThresholdImageFilterPython:

binary_threshold_image_filter(*args, **kwargs)
        Binarize an input image by thresholding.
    
        This filter produces an output image whose pixels are either one of
        two values ( OutsideValue or InsideValue ), depending on whether the
        corresponding input image pixels lie between the two thresholds (
        LowerThreshold and UpperThreshold ). Values equal to either threshold
        is considered to be between the thresholds.
    
        More precisely \\[ Output(x_i) = \\begin{cases} InsideValue &
        \\text{if $LowerThreshold \\leq x_i \\leq UpperThreshold$}
        \\\\ OutsideValue & \\text{otherwise} \\end{cases} \\]
    
        This filter is templated over the input image type and the output
        image type.
    
        The filter expect both images to have the same number of dimensions.
    
        The default values for LowerThreshold and Up

In [15]:
img_thresh = itk.binary_threshold_image_filter(img, upper_threshold=155, inside_value=0, outside_value=1) #TODO: inside_value=0,outside_value=1 don't work here
view(image=img, label_map=img_thresh)
#compare(img, img_thresh, link_cmap=True)

Viewer(geometries=[], gradient_opacity=0.22, interpolation=False, point_sets=[], rendered_image=<itk.itkImageP…

Viewer(geometries=[], gradient_opacity=0.22, interpolation=False, point_sets=[], rendered_label_map=<itk.itkIm…

In [43]:
img_thresh = itk.iso_data_threshold_image_filter(img,) #TODO: why this operation made data 2^15 ?
view(img_thresh,cmap=itkwidgets.cm.BrBG)

Viewer(cmap='BrBG', geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itkImagePython.itkIma…

In [17]:
compare(img,img_thresh) #todo: only second panel works

AppLayout(children=(HBox(children=(Label(value='Link:'), Checkbox(value=False, description='cmap'), Checkbox(v…